In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/Training Files/merged training data.zip' -d '/content/'

In [ ]:
!unzip '/content/drive/My Drive/Alankrita/Morphology task/Saved Models/model_CV.zip' -d '/content/models'

# Data Pre-Processing

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
import imgaug as ia
from random import shuffle
import cv2
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model, load_model
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.layers.pooling import _GlobalPooling1D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.nasnet import NASNetMobile, NASNetLarge
from glob import glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

In [ ]:
import pandas as pd
import os
import random
import cv2
from sklearn.utils import shuffle
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook



In [ ]:
def path(row):
    
    return '/content/train/' + row.id 

In [ ]:
def get_id(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Merged Train labels/train_labels.csv")
id_label_map = {k:v for k,v in zip(train.id.values, train.label.values)}
train.head(15)

In [ ]:
train["path"] = train.apply(path, axis=1)

In [ ]:
train.head(10)

In [ ]:
train['index'] = -1

In [ ]:
stratkf = StratifiedKFold(n_splits=3, random_state=8, shuffle=True)

i = 0

for train_index, test_index in stratkf.split(train, train['label']):
    train.loc[test_index, 'index'] = i
    
    i+=1

In [ ]:
ind = 2

len(train[(train['index'] == ind) & (train['label'] == 1)])

In [ ]:
train

In [ ]:
lab_files = glob('/content/train/*.png')
print("Labeled files size :", len(lab_files))

# Data Augmentation

In [ ]:
def slicer(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
def get_seq():
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    sequential = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.2), # vertically flip 20% of all images
            sometimes(iaa.Affine(
                #scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
                rotate=(-90, 90), # rotate by -45 to +45 degrees
                shear=(-5, 5), # shear by -16 to +16 degrees
                order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            # execute 0 to 5 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 5),
                [
                    sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
                        iaa.AverageBlur(k=(3, 5)), # blur image using local means with kernel sizes between 2 and 7
                        iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), # sharpen images
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                    # search either for all edges or for directed edges,
                    # blend the result with the original image using a blobby mask
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5), # add gaussian noise to images
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.05), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
                    ]),
                    iaa.Invert(0.01, per_channel=True), # invert color channels
                    iaa.Add((-2, 2), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-1, 1)), # change hue and saturation
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.OneOf([
                        iaa.Multiply((0.9, 1.1), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-1, 0),
                            first=iaa.Multiply((0.9, 1.1), per_channel=True),
                            second=iaa.ContrastNormalization((0.9, 1.1))
                        )
                    ]),
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                    sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    return sequential

def data_gen(list_files, id_label_map, batch_size, augment=False):
    sequential = get_seq()
    while True:
        shuffle(list_files)
        for batch in slicer(list_files, batch_size):
            X = [cv2.imread(x) for x in batch]
            Y = [id_label_map[get_id(x)] for x in batch]
            if augment:
                X = sequential.augment_images(X)
            X = [preprocess_input(x) for x in X]
                
            yield np.array(X), np.array(Y)
    

# Selective Augmentation

In [ ]:
for i in range(3):
   train_position = train[(train['index'] != i) & (train['label'] == 1)]
   train_only = train[train['index'] != i]
   train_enhance = pd.concat([train_only,train_position,train_position,train_position,train_position,train_position, train_position, train_position], ignore_index=True)
   traindata = train_enhance.loc[train_enhance['index'] != i , 'path']
   val = train.loc[train['index'] == i , 'path']
    
   print("CV: " + str(i))
   print("Train: " + str(len(traindata)))
   print("Val: " + str(len(val)))

In [ ]:
images = data_gen(train.loc[train['index'] == 0 , 'path'], id_label_map, batch_size=18, augment=True)
plt.figure(figsize=(15,10))

for (image, y) in images:
    for i, _im in enumerate(image):
        plt.subplot(3,6,i+1)
        plt.imshow((_im-_im.min())/ (_im.max() - _im.min()))
        plt.axis('off')
    break
    
plt.show()

# Fitting and training the model architecture

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2

def model_resnet(): 
    inputs = Input((130, 130, 3))
    b_model = ResNet50V2(include_top=False, input_shape=(130, 130, 3),weights='imagenet') #CHANGE
    x = b_model(inputs)
    op1 = GlobalMaxPooling2D()(x)
    op2 = GlobalAveragePooling2D()(x)
    op3 = Flatten()(x)
    out = Concatenate(axis=-1)([op1, op2, op3])
    out = Dropout(0.5)(out)
    out = Dense(1, activation="sigmoid", name="3_")(out)
    model = Model(inputs, out)
    model.compile(optimizer=Adam(0.00001), loss=binary_crossentropy, metrics=['acc'])
    print("Model compiled")
    #model.summary()

    return model

**Note:** **The model was trained using 10 epochs for 3 Cross Validation splits each, due to logistical insufficiency. However, it is recommended to the evaluators to uncomment all entire section and then train the model to get the original planned out results.** 

In [ ]:
for i in range(3):

    model = model_resnet()
    
    batch_size=64
    h5_path = "/content/modelz" + str(i) + ".h5"
    checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

    history = model.fit_generator(
        data_gen(traindata, id_label_map, batch_size, augment=True),
        validation_data=data_gen(val, id_label_map, batch_size),
        epochs=10, verbose=1,
        callbacks=[checkpoint],
        steps_per_epoch=len(traindata) // batch_size,
        validation_steps=len(val) // batch_size)
    batch_size=24
    history = model.fit_generator(
        data_gen(traindata, id_label_map, batch_size, augment=True),
        validation_data=data_gen(val, id_label_map, batch_size),
        epochs=50, verbose=1,
        callbacks=[checkpoint],
        steps_per_epoch=len(traindata) // batch_size,
        validation_steps=len(val) // batch_size)
    batch_size=32
    history = model.fit_generator(
        data_gen(traindata, id_label_map, batch_size, augment=True),
        validation_data=data_gen(val, id_label_map, batch_size),
        epochs=80, verbose=1,
        callbacks=[checkpoint],
        steps_per_epoch=len(traindata) // batch_size,
        validation_steps=len(val) // batch_size)
    del(model)
    del(traindata)
    del(val)
    print("Model erased")
    

#model.load_weights(h5_path)

 Now we predict each validation set on the respective model to find the best threshold

In [ ]:
from glob import glob
from tqdm import tqdm

# Quality Score Calculation

In [ ]:
!wget https://www.dropbox.com/s/4uwo0u0zbywbdpa/Sperm-Data.zip?dl=0

In [ ]:
!unzip '/content/Sperm-Data.zip?dl=0' -d '/content/'

In [ ]:
test_folders = glob("/content/Sperm-Data/Sperm - Unlabeled/*/")
test_folders

In [ ]:
from glob import glob
files = glob("/content/Sperm-Data/Sperm - Unlabeled/*/*/*/*")
files

In [ ]:
# Here we use one of the models to predict the unlabeled data and calculate the quality score.

model.load_weights(h5_list[-3])

for _test in test_folders:
    preds = []
    ids = []
    files = glob("/content/Sperm-Data/Sperm - Unlabeled/*/*/*/*")
    for batch in slicer(files, batch_size):
        
        X = [preprocess_input(cv2.resize(cv2.imread(x), (130,130))) for x in batch]
        ids_batch = [get_id(x) for x in batch]
        X = np.array(X)
        preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
        preds += preds_batch
        ids += ids_batch
    preds = np.array(preds)
    preds = 1 * (preds > 0.29) # Make sure to change this threshold according to the value in the prior cells. Get the Thr number and divide by 1000 
    print(_test.split("/")[-2] + ": " + str(int(100 * preds.sum()/len(preds)))) #formula to calculate
    df = pd.DataFrame({'id':ids, 'label':preds})
    df.to_csv("result_" + _test.split("/")[-2] + ".csv", index=False)


In [ ]:
from tqdm import tqdm

# Model Speculation

In [ ]:


model = get_model_classif_densenet()

h5_list = sorted(glob("/content/models/*"))# h5_list

for i, h5_path in enumerate(tqdm(h5_list[-3:])):
    model.load_weights(h5_path)
    val = df_train.loc[df_train['index'] == i , 'path']
    preds = []
    ids = []
    for batch in slicer(val, batch_size):
        X = [preprocess_input(cv2.imread(x)) for x in batch]
        ids_batch = [get_id_from_file_path(x) for x in batch]
        X = np.array(X)
        preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
        preds += preds_batch
        ids += ids_batch
    df = pd.DataFrame({'id':ids, 'label':preds})
    df_merge = pd.merge(df, df_train, how='left', on='id')
    df_merge[['label_x', 'label_y']].to_csv('res_crossval' + str(i) + '.csv')

In [ ]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


F1-Scores - Validation Inference

In [ ]:
best_th = []
for i in range(3):
    df = pd.read_csv('/content/res_crossval' + str(i) + '.csv')
    y_test = df['label_y'].values
    y_score = df['label_x'].values
    f1 = []
    for _t in range(0, 1001, 1):
        th = _t / 1000
        f1.append(f1_score(y_test, y_score > th, ))
    f1 = np.array(f1)
    print("F1 score: " + str(f1[f1.argmax()]) + " Threshold: " + str(f1.argmax()))
    

In [ ]:
df = pd.read_csv("/content/res_crossval0.csv")
df1 = pd.read_csv("/content/res_crossval1.csv")
df2 = pd.read_csv("/content/res_crossval2.csv")



Test Acc_CV0

In [ ]:
n_classes = 1
y_test = df["label_y"].values
y_score = df["label_x"].values

fpr = dict()
tpr = dict()
roc_auc = dict()

fpr, tpr, thr = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# Compute micro-average ROC curve and ROC area
fpr, tpr, thr = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
#Plot of a ROC curve for a specific class
roc_auc

In [ ]:
acc = []
for _t in thr:
    acc.append(accuracy_score(y_test, y_score > _t))

acc = np.asarray(acc)
acc[acc.argmax()]


Best Thr_CV0

In [ ]:
best_thr = thr[acc.argmax()]
best_thr

In [ ]:
confusion_matrix(y_test, y_score > best_thr)

Inference on CV1 - ROC_AUC

Test Acc_CV1

In [ ]:
n_classes = 1
y_test = df1["label_y"].values
y_score = df1["label_x"].values

fpr = dict()
tpr = dict()
roc_auc = dict()

fpr, tpr, thr = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# Compute micro-average ROC curve and ROC area
fpr, tpr, thr = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
#Plot of a ROC curve for a specific class
roc_auc

In [ ]:
acc = []
for _t in thr:
    acc.append(accuracy_score(y_test, y_score > _t))

acc = np.asarray(acc)
acc[acc.argmax()]


Best_thr_CV1

In [ ]:
best_thr = thr[acc.argmax()]
best_thr

In [ ]:
confusion_matrix(y_test, y_score > best_thr)

Inference on CV2 - ROC_AUC

Test_ACC - CV2

In [ ]:
n_classes = 1
y_test = df2["label_y"].values
y_score = df2["label_x"].values

fpr = dict()
tpr = dict()
roc_auc = dict()

fpr, tpr, thr = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# Compute micro-average ROC curve and ROC area
fpr, tpr, thr = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
#Plot of a ROC curve for a specific class
roc_auc

In [ ]:
acc = []
for _t in thr:
    acc.append(accuracy_score(y_test, y_score > _t))

acc = np.asarray(acc)
acc[acc.argmax()]


Best_thr_CV2

In [ ]:
best_thr = thr[acc.argmax()]
best_thr

In [ ]:
confusion_matrix(y_test, y_score > best_thr)